Import thư viện

In [2]:
import re
import os
import underthesea
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from gensim.models import Word2Vec
from tensorflow.keras import layers, models
import gensim.models.keyedvectors as word2vec

# 1. Thu thập và phân tích đặc điểm dữ liệu

- score < 4 hoặc NaN: Negative
- 4 < score < 7 : Neutral
- score > 7 : Positive

độ dài ngắn của các bình luận giao động:
min=1, max > 3800 từ / 1 bình luận

# 2. Tiền xử lý dữ liệu

sử dụng một số tiền xử lý cơ bản như:
- Chuẩn hóa về chữ thường
- Thay thế các url trong dữ liệu bởi nhãn link_spam
- Tách từ (Sử dụng underthesea)
- Loại bỏ dấu câu và các ký tự đặc biệt
- Xử lý các trường hợp người dùng láy láy âm tiết(Ví dụ: Ngooon quááááá !!!!!)
- Chuẩn hóa các từ viết tắt cơ bản(Ví dụ: k, ko, k0, khong, khôg --> không, bt --> bình thường,...)
- Loại bỏ số và các từ chỉ có 1 ký tự

In [3]:
def preprocess_text(text):
    # Chuẩn hóa về chữ thường
    text = text.lower()
    # Thay thế các URL bằng nhãn 'link_spam'
    text = re.sub(r'http\S+', 'link_spam', text)
    # Tách từ
    words = underthesea.word_tokenize(text)
    # Loại bỏ dấu câu và các ký tự đặc biệt
    words = [re.sub(r'[^\w\s]', '', word) for word in words]
    # Xử lý các trường hợp người dùng lặp lại âm tiết
    words = [re.sub(r'(\w)\1+', r'\1', word) for word in words]
    # Chuẩn hóa các từ viết tắt cơ bản
    abbreviation_dict = {
        'k': 'không',
        'ko': 'không',
        'k0': 'không',
        'khong': 'không',
        'khôg': 'không',
        'bt': 'bình thường'
        # Thêm các từ viết tắt cần chuẩn hóa
    }
    words = [abbreviation_dict.get(word, word) for word in words]
    # Loại bỏ số và các từ chỉ có 1 ký tự
    words = [word for word in words if not (word.isdigit() or len(word) == 1)]
    # Ghép lại các từ thành câu
    cleaned_text = ' '.join(words)
    
    return cleaned_text

Test xử lí data mẫu

In [4]:
# Ví dụ sử dụng hàm preprocess_text
text = "Ngooon quááááá !!!!!"
cleaned_text = preprocess_text(text)
print(cleaned_text)

ngon quá     


# 3. Vector hóa dữ liệu

Word Embedding

In [4]:
# Đọc File
path = './data/'
def readdata(path):
    list_file = os.listdir(path)
    data = pd.DataFrame()
    for filename in list_file:
        try:
            # Đọc file CSV 
            data = pd.concat([data, pd.read_csv(os.path.join(path, filename), sep=',', encoding='latin1')])
        except pd.errors.ParserError:
            # Skip nếu ko đúng định dạng file
            print(f"Error reading file: {os.path.join(path, filename)}. Skipping...")
    return data.Review, data.Label

Đưa dữ liệu về định dạng input của gensim

In [5]:
reviews, labels = readdata(path)
input_gensim = [review.split() for review in reviews]
for review in reviews:
    input_gensim.append(review.split())
    
model = Word2Vec(input_gensim, vector_size=128, window=5, min_count=0, workers=4, sg=1)
model.wv.save("word.model")

Error reading file: ./data/negative_data.xlsx. Skipping...
Error reading file: ./data/neural_data.xlsx. Skipping...
Error reading file: ./data/positive_data.xlsx. Skipping...


Nhúng các từ trong một bình luận thành ma trận nhúng 

In [6]:
model_embedding = word2vec.KeyedVectors.load('./word.model')

word_labels = []
max_seq = 200
embedding_size = 128

for word in model_embedding.index_to_key:
    word_labels.append(word)
    
def comment_embedding(comment):
    matrix = np.zeros((max_seq, embedding_size))
    words = comment.split()
    lencmt = len(words)

    for i in range(max_seq):
        indexword = i % lencmt
        if (max_seq - i < lencmt):
            break
        if(words[indexword] in word_labels):
            matrix[i] = model_embedding[words[indexword]]
    matrix = np.array(matrix)
    return matrix

# 4. Xây dựng và huấn luyện mô hình

Tiến hành số hóa tất cả dữ liệu đầu vào, các câu comment được chuyển về ma trận số, các label được chuyển về dạng one-hot.

[1, 0, 0] : neutral

[0, 1, 0] : positive

[0, 0, 1] : negative

In [7]:
train_data = []
label_data = []

for x in tqdm(reviews):
    train_data.append(comment_embedding(x))
train_data = np.array(train_data)

for y in tqdm(labels):
    label_ = np.zeros(3)
    try:
        label_[int(y)] = 1
    except:
        label_[0] = 1
    label_data.append(label_)

100%|██████████| 9727/9727 [00:00<00:00, 981948.47it/s]


định nghĩa các tham số cho mô hình:

In [8]:

sequence_length = 200
embedding_size = 128
num_classes = 3
filter_sizes = 3
num_filters = 150
epochs = 50
batch_size = 30
learning_rate = 0.01
dropout_rate = 0.5

Định nghĩa kiến trúc mô hình:

In [9]:
# Reshape input data
x_train = train_data.reshape(train_data.shape[0], sequence_length, embedding_size, 1).astype('float32')
y_train = np.array(label_data)

# Định nghĩa model
model = models.Sequential()
model.add(layers.Conv2D(num_filters, (filter_sizes, embedding_size),
                        padding='valid',
                        input_shape=(sequence_length, embedding_size, 1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(sequence_length - filter_sizes + 1, 1)))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

# Train model
adam = tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])
print(model.summary())


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 1, 150)    │        57,750 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 1, 1, 150)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 1, 150)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        19,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,465 (302.60 KB)

 Trainable params: 77,465 (302.60 KB)

 Non-trainable params: 0 (0.00 B)

None


Tiến hành train mô hình, sử dụng 7000 sample đầu để train, còn lại cho validaion.

In [10]:
model.fit(x = x_train[:7000], y = y_train[:7000], batch_size = batch_size, verbose=1, epochs=epochs, validation_data=(x_train[:3000], y_train[:3000]))
model.save('models.h5')

Epoch 1/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.5627 - loss: 0.9329 - val_accuracy: 0.6813 - val_loss: 0.8529
Epoch 2/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.6878 - loss: 0.7430 - val_accuracy: 0.7180 - val_loss: 0.7555
Epoch 3/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.7122 - loss: 0.6895 - val_accuracy: 0.7397 - val_loss: 0.7023
Epoch 4/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.7316 - loss: 0.6462 - val_accuracy: 0.7977 - val_loss: 0.5750
Epoch 5/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.7439 - loss: 0.6053 - val_accuracy: 0.8010 - val_loss: 0.5579
Epoch 6/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.7609 - loss: 0.5816 - val_accuracy: 0.8277 - val_loss: 0.4678
Epoch 7/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.7743 - loss: 0.5497 - val_accuracy: 0.8233 - val_loss: 0.5134
Epoch 8/50
234/234 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.7849 - loss: 0.5264 - val_accu

Lưu mô hình đã được huấn luyện

In [11]:
model.save('models.h5')

Sau khi train model thì sẽ lưu model lại để sử dụng

# 5. Kiểm thử mô hình dự đoán

Load lại mô hình

In [12]:
from keras.models import load_model
model_sentiment = load_model("models.h5")

ví dụ 1 mẫu để kiểm thử mô hình

In [13]:

text = "đồ ăn ở đây vừa nhiều vừa ngon"
text = preprocess_text(text)

maxtrix_embedding = np.expand_dims(comment_embedding(text), axis=0)
maxtrix_embedding = np.expand_dims(maxtrix_embedding, axis=3)

result = model.predict(maxtrix_embedding)
result = np.argmax(result)
print("Label predict: ", result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Label predict:  1


Kí hiệu Label predict:
- 1 : Positive
- 0 : Neutral
- 2 : Negative